In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('gb.csv')

In [4]:
data.iloc[1:-1].std()

mean_delay        0.026334
count         39905.733780
dtype: float64

In [7]:
data

,hour,mean_delay,count
0,2020.04.30 00,0.338620,215437
1,2020.04.30 01,0.241578,214455
2,2020.04.30 02,0.243426,233508
3,2020.04.30 03,0.237617,160895
4,2020.04.30 04,0.243973,214689
5,2020.04.30 05,0.231458,159229
6,2020.04.30 06,0.234219,273332
7,2020.04.30 07,0.215542,222348
8,2020.04.30 08,0.227765,280779
9,2020.04.30 09,0.232021,225161


In [16]:
def make_time_features(data, timestamps=[24, 12, 6, 5, 4, 3, 2, 1], target='mean_delay'):
    targets = []
    features = []
    for i in range(data.shape[0]):
        temp_arr = []
        for s in timestamps:
            temp = data.shift(s)[target].loc[i]
            temp_arr.append(temp)
            
        targets.append(data[target].loc[i])
        features.append(temp_arr)
    
    res_df = pd.DataFrame(features)
    res_df['target_{}'.format(target)] = targets
    
    rename_dict = {}
    for i in range(len(timestamps)):
        rename_dict[i] = 'lag_{}_hours'.format(timestamps[i])
    res_df = res_df.rename(columns=rename_dict)
    
    return res_df
    
    

In [10]:
import numpy as np

In [97]:
def make_mean_features(data, timestamps=[24, 12, 6, 5, 4, 3, 2], target='mean_delay'):
    target_column = data['target_{}'.format(target)]
    features = []
    for timestamp in timestamps:
        column = [np.nan] * (timestamp-1)
        for i in range(timestamp, len(data)):
            data_slice = target_column[i-timestamp:i]
            column.append(np.mean(data_slice))
        features.append(column)
        
    res_df = pd.DataFrame(np.array(features).T, columns=['mean_{}_hours'.format(timestamps[j])
                          for j in range(len(timestamps))])
    #res_df['target_{}'.format(target)] = target_column
    return res_df

In [126]:
lags = make_time_features(data, target='mean_delay')

X = make_mean_features(lags, target='mean_delay')

X = pd.concat((X, d), axis=1)

y = X.target_mean_delay
X = X.fillna(0).drop('target_mean_delay', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [127]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [128]:
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

class TargetForecasting(object):
    def __init__(self, estimator='cat', **params):
        if estimator == 'cat':
            self.estimator = CatBoostRegressor(**params)
        elif estimator == 'rf':
            self.estimator = RandomForestRegressor(**params)
        elif estimator == 'lin':
            self.estimator = LinearRegression(**params)
        
    def fit(self, X, y, **fit_params):
        self.estimator.fit(X, y, **fit_params)
    
    def predict(self, X):
        return self.estimator.predict(X)

In [131]:
model = TargetForecasting(estimator='rf', n_estimators=40)

model.fit(X_train, y_train)

In [133]:
mean_absolute_error(y_test, model.predict(X_test))

0.03629709975222744